In [3]:
import requests
import json

import pandas as pd
from datetime import datetime

headers = {
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'no-cache',
    'origin': 'https://edition.cnn.com',
    'pragma': 'no-cache',
    'priority': 'u=1, i',
    'referer': 'https://edition.cnn.com/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
}

response = requests.get('https://production.dataviz.cnn.io/index/fearandgreed/graphdata/2021-01-01', headers=headers)

with open('../datasets/fear_greed.json', 'w') as f:
    json.dump(response.json(), f, indent=4)


In [4]:
with open('../datasets/fear_greed.json', 'r') as file:
    data = json.load(file)

fear_and_greed_data = data.get('fear_and_greed_historical', {}).get('data', {})

for entry in fear_and_greed_data:
    timestamp = entry['x'] / 1000  # Convert milliseconds to seconds
    entry['x'] = datetime.utcfromtimestamp(timestamp).strftime('%d-%m-%Y')


extracted_data = [{'Date': entry['x'], 'rating': entry['rating']} for entry in fear_and_greed_data]

df = pd.DataFrame(extracted_data)

df = df[:-1]
df.to_csv('../datasets/historical_prices.csv', index=False)

In [5]:
import yfinance as yf
import pandas as pd

historical_prices = pd.read_csv("../datasets/historical_prices.csv")
historical_prices['Date'] = pd.to_datetime(historical_prices['Date'], format='%d-%m-%Y')
historical_prices.set_index('Date', inplace=True)

tickers = {
    "GC=F": "Gold",
    "CL=F": "Crude Oil",
    "PL=F": "Platinum",
    "DX-Y.NYB": "USD Index"
}

for ticker, column_name in tickers.items():
    data = yf.download(ticker, start="2021-01-01", end="2024-09-20")
    open_prices = data['Open']
    df = pd.DataFrame(open_prices)
    df.columns = [column_name]
    
    historical_prices = historical_prices.join(df, how='outer')

# Save the combined data to a CSV file
historical_prices.to_csv("../datasets/historical_prices.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
